In [3]:
    import fiona
    import os,os.path
    from shapely.geometry import shape,mapping
    from shapely.ops import transform
    from functools import partial
    import pyproj
    from fiona.crs import from_epsg
    import osmnx as ox
    import matplotlib.pyplot as plt
    import geopandas as gpd
    import glob
    import json
    import numpy as np
    import requests as rq
    import collections
    import logging
    import time
    import dateutil.parser
    %matplotlib inline

In [2]:
# Create a custom logger
logger = logging.getLogger(__name__)

# Create handlers
f_handler = logging.FileHandler('Mapillary.log')
f_handler.setLevel(logging.DEBUG)

# Create formatters and add it to handlers
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(f_handler)

In [3]:
mapillary_Image_ID = 'https://a.mapillary.com/v3/images?client_id=OFFJd0ZNdzdWY0lkVVpyc2kyZThBZzpkYmIzYTVkYTgxMDBmYmY0&closeto='
distanceFrag = '&radius=200'
mapillary_Image_url = 'https://images.mapillary.com/' #<image_key>/thumb-640.jpg

In [4]:
city = '../Data/GreaterLondon/'
files= glob.glob(city + '*.json')

In [ ]:
for f in files:
    name = 'Mapillary_'+'_'.join(f.split('/')[-1].split('_')[-2:])
    logger.debug("Working with %s"%(f))
    first = json.load(open(f , 'rb'))
    points = []
    for k in first:
        comps = k.split('_')
        point = (comps[1].strip(),comps[0].strip())
        points.append(point)
    logger.debug('Found %d points in this file'%(len(points)))
    
    imagesMeta = []
    for point in points:
        loc= point[0]+','+point[1]
        finalUrl = mapillary_Image_ID + loc + distanceFrag
        try:
            response = rq.get(finalUrl,timeout=5)
            if response.status_code == 200:
                dictionary = json.loads(response.content)
                imagesMeta += dictionary["features"]
            else:
                logger.error('Missed point at (%s,%s)'%(point[0],point[1]))
                time.sleep(10)
        except:
            logger.error('Request Timed out')
    logger.debug('Found %d Images in this file'%(len(imagesMeta)))
    logger.debug('Dumping in file: %s'%(name))
    json.dump(imagesMeta,open(city+name,'wb'))